# funcion para obtener informacion sin limite de datos

In [9]:
import requests
import json
from requests.auth import HTTPBasicAuth
import time
import pandas as pd
import re

## Prueba Github

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

def elasticScroll(elasticParameters, query, pages):
    # parametros de salida
    # parametros del indice
    elasticURL = elasticParameters["elasticURL"]
    elasticIndex = elasticParameters["elasticIndex"]
    elasticUser = elasticParameters["elasticUser"]
    elasticPassword = elasticParameters["elasticPassword"]
    
    if(len(elasticURL)==0) or (len(elasticIndex)==0) or (len(elasticUser)==0) or (len(elasticPassword)==0):
        raise Exception("Revisa los parametros")
    # se define la url que apunta al indice de elastic
    url_search = f"{elasticURL}/{elasticIndex}/_search?scroll=1m"
    # se ejecuta la query
    response = requests.get(url_search, json=query, auth=HTTPBasicAuth(elasticUser, elasticPassword))
    # retorna una lista con el resultado de la query
    search = json.loads(response.text)
    # guardamos el scroll id correspondiente a la query
    scroll_id = search["_scroll_id"]
    # url scroll
    url_scroll = f"{elasticURL}/_search/scroll"
    scroll_query = {
                "scroll": "1m",
                "scroll_id": f"{scroll_id}"
            }

    # condiciones iniciales
    scroll_search = {"hits":{"hits":1}}
    if pages:
        # hay paginacion
        # condiciones iniciales
        from_ = pages["from"]
        size_ = pages["size"]
        count = len(search["hits"]["hits"])

        while scroll_search["hits"]["hits"] and count < from_ + size_:
            scroll_response = requests.get(url_scroll, json=scroll_query, auth=HTTPBasicAuth(elasticUser, elasticPassword))
            scroll_search = json.loads(scroll_response.text)
            if not scroll_search["hits"]["hits"]:
                continue
            else:
                search["hits"]["hits"].extend(scroll_search["hits"]["hits"])
            count += len(scroll_search["hits"]["hits"])

        search["hits"]["hits"] = search["hits"]["hits"][from_:from_+size_+1]

    else:
        # Se devuelven todos los resultados
        while scroll_search["hits"]["hits"]:
            scroll_response = requests.get(url_scroll, json=scroll_query,
                                           auth=HTTPBasicAuth(elasticUser, elasticPassword))
            scroll_search = json.loads(scroll_response.text)
            if not scroll_search["hits"]["hits"]:
                continue
            else:
                search["hits"]["hits"].extend(scroll_search["hits"]["hits"])
    # Se elina el campo scroll del scroll_body
    del scroll_query["scroll"]
    # Se elimina el scroll de elasticsearch para liberar memoria
    delete = requests.delete(url_scroll, json=scroll_query, auth=HTTPBasicAuth(elasticUser, elasticPassword))
    return search

In [10]:
# definiendo parametros de entrada

In [11]:
# datos de conexion a elastic
elasticParameters = {"elasticURL": "https://es-dev.e-contact.cl"
                    , "elasticIndex": "lea_sequences-events-banco_de_chile" 
                    , "elasticUser": "jcalderon"
                    , "elasticPassword": "jcalderon123"
                    }

# query custom para obtener fechas
query = {
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        },
        { 
            
          "range": {
            "interactionData.dateTimeUTC": {
              "format": "strict_date_optional_time",
              "gte": "2022-05-28T00:00:00.000Z",
              "lte": "2022-05-29T00:00:00.000Z"
            }
          }
         # "interactionData.interactionId" :{"7878d505-9ef5-4e6c-afcd-76b5f2834562"}  
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}

# total de paginas
pages =  {
    "from": 0,
    "size": 5000
}

# consulta a indice

In [12]:
response = elasticScroll(elasticParameters, query, pages)

In [13]:
def limpiar_texto(texto):
    """
    Función para realizar la limpieza de un texto dado.
    """
    # Eliminamos los caracteres especiales
   # texto = re.sub(r'\W', ' ', texto)
    # Eliminado las palabras que tengo un solo caracter
    texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', str(texto))
    # Sustituir los espacios en blanco en uno solo
    texto = re.sub(r'\s+', ' ', texto, flags=re.I)
    replacements = (
        ("á", "a"),("é", "e"),("í", "i"),("ó", "o"),("ú", "u"),
        ("ñ", "n"),("?", ""),("¿", ""),("%", ""),("$", ""),
        ("#", ""),("&", ""),("(", ""),(")", ""),("=", ""),
        ("¡", ""),("!", ""),("*", ""),("+", ""),("~", ""),
        ("[", ""),("]", ""),("}", ""),("{", ""),("^", ""),
        ("<", ""),(">", ""),("¬", ""),("¨", ""),("_", "")
    )
    
    for a, b in replacements:
        texto = texto.replace(a, b)
    # Convertimos textos a minusculas
    #texto = texto.lower()
    return texto

# transformar response (dict) a dataframe

In [14]:
from pandas import DataFrame, json_normalize
import numpy as np

df = json_normalize(json.loads(json.dumps(response))["hits"]["hits"])
df_00 = df [['_source.conversationSequence.topicName','_source.interactionData.interactionId']]

# renombra columnas
df_00['topicName']= df_00['_source.conversationSequence.topicName']
df_00['interactionId']= df_00['_source.interactionData.interactionId']

# reemplaza valores vacios por "NO IDENTIFICADO"
df_00["topicName"] = df_00["topicName"].replace({ "": "NO_IDENTIFICADO", " ": "NO_IDENTIFICADO" })


df_00['topicName'] = df_00.topicName.apply(limpiar_texto)

# elimina duplicados
df_00 = df_00.drop_duplicates()

# crea df 01 agrupando y contando por columna interactionId
df_01=df_00.groupby(["interactionId"]).size().reset_index(name='cantidad')

# concatena ambos df (00 y 01) para crear df 02
df_02 = df_00.merge(df_01, how='inner', on='interactionId')
#######################################################################

# filtro para aplicar en el df 02 cuando el campo cantidad == 1
filter01 = df_02["cantidad"]==1
df_02.where(filter01, inplace = True)

# crea df 03 apartando aquellos registros Nan
df_03 = df_02[df_02['cantidad'].notna()]

##########################################################################

# crea df 04 agrupando y contando por columna interactionId
df_04=df_00.groupby(["interactionId"]).size().reset_index(name='cantidad')

# concatena ambos df (00 y 04) para crear df 05
df_05 = df_00.merge(df_04, how='inner', on='interactionId')

# filtro para aplicar en el df 05 cuando el campo cantidad != 1
filter02 = df_05["cantidad"]!=1
df_05.where(filter02, inplace = True)

# crea df 06 apartando aquellos registros Nan
df_06 = df_05[df_05['cantidad'].notna()]

# crea df  apartando aquellos registros cuyo topicName == 'NO_IDENTIFICADO'
new_df = df_06[~df_06['topicName'].str.contains("NO_IDENTIFICADO")]

# concatena ambos df (03 y new_df) para crear df 01
df_01 = pd.concat([df_03, new_df], axis=0)

# elimina col cantidad
del df_01['cantidad']

# crea df "primario" con valores que corresponden al grafo 02
df_grafo2 = pd.DataFrame({'topicName': ['Corte llamado', 'Red sucursal','Reclamación','Escalamiento de llamada','Reincidencia'], 'matched': True})

# agrupa y cuenta los topicos 
df_02=df_01.groupby(["topicName"]).size().reset_index(name='cantidad')

# agrega columna total contabilizando el total de interacciones obtenidas en el primer query
df_02['Total'] = df_02['cantidad'].sum()

# crea df haciendo match entre el total de topicos y aquellos que utiliza el grafo 02
df_03 = df_02.merge(df_grafo2, how='left', on='topicName')

# selecciona solo aquellos topicos del grafo 01
df_03 = df_03[pd.isnull(df_03['matched'])]

# crea df haciendo match entre el total de topicos y aquellos que utiliza el grafo 02
df_04 = df_02.merge(df_grafo2, how='inner', on='topicName')

# reduce cols a utilizar en los df
df_grafo01 = df_03[['topicName','cantidad','Total']]
df_grafo02 = df_04[['topicName','cantidad','Total']]

# calcula el % dividiendo cantidad de apariciones por el total de interacciones
df_grafo01['Final'] =((df_grafo01['cantidad']) / df_grafo01['Total'])
df_grafo02['Final'] =((df_grafo02['cantidad']) / df_grafo02['Total'])

# col grafico determina a cual grafica se deben utilizar los datos
df_grafo01['Grafico']=1
df_grafo02['Grafico']=2

# fusiona y crea solo un df
result = pd.concat([df_grafo01, df_grafo02], axis=0)

# formatea salidas segun lo solcitado
result = result.set_index('topicName').T.to_dict('list')

result
#df_00

{'Activacion de productos': [5.0, 127.0, 0.03937007874015748, 1.0],
 'Baja de productos': [1.0, 127.0, 0.007874015748031496, 1.0],
 'Clave Digipass': [4.0, 127.0, 0.031496062992125984, 1.0],
 'Clave Internet': [2.0, 127.0, 0.015748031496062992, 1.0],
 'Consulta compras TD TC': [5.0, 127.0, 0.03937007874015748, 1.0],
 'Consulta ejecutivo de cuenta': [1.0, 127.0, 0.007874015748031496, 1.0],
 'Consulta estado de cuenta facturacion': [3.0,
  127.0,
  0.023622047244094488,
  1.0],
 'Consulta inversiones': [1.0, 127.0, 0.007874015748031496, 1.0],
 'Consulta por cheques': [2.0, 127.0, 0.015748031496062992, 1.0],
 'Desbloqueo Clave': [4.0, 127.0, 0.031496062992125984, 1.0],
 'Emergencias bancarias': [12.0, 127.0, 0.09448818897637795, 1.0],
 'NOIDENTIFICADO': [70.0, 127.0, 0.5511811023622047, 1.0],
 'Oportunidades de vinculacion': [2.0, 127.0, 0.015748031496062992, 1.0],
 'Problemas con plastico': [1.0, 127.0, 0.007874015748031496, 1.0],
 'Problemas uso tarjetas': [1.0, 127.0, 0.007874015748031